## Solving Common Introductory Quantum Mechanics Problems using Sympy



S. Singleton, inspired by Erica Harvey of the
[Symmath Project](https://pubs.acs.org/doi/pdf/10.1021/ed081p155).

This notebook show basic examples for quantum calculations encountered
in undergraduate physical chemistry course. The purpose of this notebook
is not to explain the theory behind the calculations, bur rather show
how python can be used for solving typical problems.

This notebook assumes [sympy](http://sympy.org) is available and some
familiarity with python lists and functions. The
[sympy tutorial](http://docs.sympy.org/latest/tutorial/index.html)
describes all of the functions used in the notebook.



### Objectives



After completing this work you should be able to use python and sympy to

1.  calculate a normalization constant
2.  calculate an expectation value and its variance
3.  determine whether a function is an eigenvalue of an operator
4.  determine if functions are orthogonal



### Prerequisites



-   basic knowledge of python and sympy
-   an installed python [scipy](http://www.scipy.org) stack
-   an installed python [sympy](http://www.sympy.org) symbolic computation
    library
-   basic understanding of eigenfuctions, normalization, orthogonality,
    and expectation values



### Python environment



First, import libaries and initialize sympy&rsquo;s pretty printing.



In [1]:
# import and initialize libraries
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sympy import *

# uncomment one of these and comment the other
init_printing() # jupyter, cocalc
#init_printing(use_latex=False) # google colaboratory

### General properties of wavefunctions



Remind yourself of some key properties of wavefunctions. These
properties (sometimes call Born&rsquo;s conditions) are generally found in a
discussion on the postulates of quantum mechanics.



## Normalizing functions



One of the postulates of quantum mechanics states we can calculate the
probability of finding a particle in a certain space if the wavefunction
is known. To compare probabilities, the *normalization condition* must
be satisfied: $$ \int_{0}^a \psi(x)^* \psi(x)\; dx = 1 $$

We start with a simple function representing the wavefunction of a
particle

$$ f(x) = N_0 e^{-ax^2/2} $$

where $N_0$ and $a$ are constants.

**Question:** Is this a valid wavefunction for the particle in a box model?



`sympy` needs help to know what the variables are in a function. We can
also include sensible assumptions about those variables if they are
known. This is not necessary, but allows us to disregard physically
impossible results (and speed up some computations).



In [1]:
a, x, N0 = symbols('a x N0', positive=True)

f = E**(-a * x**2 / 2)
f

`f` is a sympy *expression* (which is different than a sympy function).

Now, define the wavefunction $\psi(x)$ using `f`:



In [1]:
psi = N0 * f
psi

**Question:** What is the difference between `f` and `psi`?



sympy&rsquo;s `conjugate()` gives the complex conjugate. Because `psi(x)` is
real, its complex conjugate is the same function.



In [1]:
# sympy conjugate() gives the complex conjugate of a function
psi_star = conjugate(psi)
psi, psi_star

##### Python interlude: complex numbers in sympy and python



In python, complex numbers (those related to $\sqrt{-1}$, the
[imaginary unit](https://en.wikipedia.org/wiki/Imaginary_unit)) and
$e^x$ can be entered in different ways. If `sympy` is loaded, `I` is the
imaginary unit and `E` is the Euler $e$. In &rsquo;plain&rsquo; python these are
`1j` and `exp()`. For example $e^{ix}$ can be entered as:

-   `E**(I*x)` or `exp(I*x)` symbolic (sympy) form

-   `exp((1j * x))` numeric (&rsquo;plain&rsquo; python) form

If in doubt, use the `type()` function to see what the computer thinks
you&rsquo;re using:



In [1]:
type(I)

In [1]:
type(1j)

## Plot the wavefunction and probability amplitude



Sympy takes a different approach to plotting than matplotlib/pyplot. The
sympy plotting functions generally produce figures that look more like
those from &ldquo;math books&rdquo;. For example, axis origin is in the center of a
figure rather than the lower left corner. Sympy assumes a plot range of
-1 < x < 1 unless told otherwise.

Use sympy to plot $\psi$ and $\psi^* \psi$, the normalized
wavefunctions.



In [1]:
# subs = substitute values for variables
psi = psi.subs({a:3, N0:norm})  # using a dictionary for multiple subsitutions
psi_star = psi_star.subs([(a,3), (N0,norm)])  # using a list

In [1]:
psi

In [1]:
p1 = plot(psi, (x, -4, 4), show=False)
p1[0].label=r'$\psi(x)$'
p1.xlabel=''
p1.ylabel=''
p1.legend=True

p2 = plot(psi**2, (x,-4,4), line_color='r', show=False) # don't show the plot until it's 'extended'
p2[0].label = r'$\psi^2(x)$'

p1.extend(p2) # combine the plot lines
p1.show()

The probability amplitude, $\psi^2$ has a narrower shape. Is $\psi$
normlized?



## Calculating probability of finding a particle in a specified region



Using the wavefunction defined above, we&rsquo;ll calculate the probability of
finding the particle in the region between x1 = 0 and x2 = 0.5.



In [1]:
x1, x2 = 0, 0.5

integrate(psi*psi_star,(x, x1, x2)).evalf()

This process might be repeated frequently. If so, it&rsquo;s probably worth
defining in the form of a python function.



In [1]:
# default limits of infinity are provided, but can be overridden
def prob_func(f, x_lower=-oo, x_upper=oo):
    'integrate the normalized wavefunction f(x) from x_l to x_u'
    
    fstar = conjugate(f)
    prob = integrate(fstar * f, (x, x_lower, x_upper))
    return prob

Print the probability over 10 intervals within the box.



In [1]:
psi = psi.subs({a:1, N0:norm})
lengths = [l/10 for l in range(0, 11)]
probs = []
print("Probability over intervals")
print(lengths)
for l in lengths:
    p = prob_func(psi, 0, l).evalf()
    probs.append(p)
    print(p)
    #print(prob_func(psi, 0, l).evalf())
Add(p)

## Calculate average values



The expectation value $\langle a \rangle$ for an observable or property
is calculated by finding the operator that describes that observable,
finding a normalized wavefunction that describes the system under
consideration, and using the integral definition for expectation value:

$$ \langle a \rangle = \int \psi^*(x)\, \hat{A}\, \psi(x)\ dx $$

The position operator is $\hat{x} = x$; just multiplication by $x$. In
our example, the range for $x$ is $-\infty$ to $\infty$, so the average
(or *expectation*) value might be predictable looking at the symmetry of
the probability plot.



In [1]:
x_avg = integrate(psi_star * x * psi, (x, -oo, oo))
x_avg

## Calculate uncertainty (standard deviation and variance) in an expectation value



Variance in a quantity can be calculated as the expectation value for
the quantity squared minus the (expectation value for the quantity)
squared. In the case of position

$$\sigma^2 = \int \psi^*\, x^2\, \psi\; dx - \left ( \int \psi^*\, x\, \psi \;dx \right )^2 $$



In [1]:
sigma_squared = integrate(psi_star * x**2 * psi, (x,-oo,oo)) - (integrate(psi_star*x*psi, (x,-oo,oo)))**2

In [1]:
sqrt(sigma_squared).evalf()

## Is a wavefunction an eigenfunction of an operator?



Eigenfunctions of an operator have the property

$$ \hat{A}f = a f$$

where $a$ is the eigenvalue.

See if the function $\psi(x)=\exp(-a_0 x^2/2)$ is an eigenfunction of
the kinetic energy operator
$$\hat{T} = \frac{\hbar^2}{2m} \frac{d^2 \psi(x)}{dx^2} $$



In [1]:
# this could be done as an expression or a function
def T_hat(f):
    "Apply the kinetic energy operator T on function f(x)"
    hbar, m = symbols('hbar m')
    return hbar**2 /(2*m) * diff(f, x, 2)

In [1]:
psi = N0 * exp(-a*x**2/2)

In [1]:
T_hat(psi)

`T_hat()` returned the result of operating on $\psi$. If $\psi$ is an
eigenfunction, we should be able to factor this result into $a\psi$
where $a$ is the eigenvalue. Another approach would be to divide both
sides by the function and see if the result is a constant.
$$\frac{\hat{A}f}{f} = a$$



In [1]:
T_hat(psi)/psi

Because this result contains $x^2$, it is not a constant. Therefore,
$\psi$ is not an eigenfunction of the kinetic energy operator $\hat{T}$.

How about $\cos(x)$?



In [1]:
T_hat(cos(x))

So, $\cos(x)$ is indeed an eigenfunction of $\hat{T}$ and the eigenvalue
is $-\hbar^2/2m$. However, $\cos(x)$ is not a valid wavefunction for the
particle-in-a-box model. Why not?



## Are two functions orthogonal?



The functions
$$\phi_n(\theta) = e^{in\theta} $$
where $n$ is an integer and $0 \le \theta \le 2\pi$ are orthogonal if

$$\int_0^{2\pi} \phi_n^*(\theta) \phi_m^*(\theta)\; d\theta = 0 $$ for
$m\ne n$.



In [1]:
var('theta n', real=True)
def phi(n): 
    return exp(I*n*theta)

integrate(conjugate(phi(1)) * phi(0), (theta, 0, 2*pi))

$\phi_1$ and $\phi_2$ are orgthogonal.



#### Python interlude: range() and loops



Using loops can make quick work of iterating over many combinations.
(NB: This is a silly example, as it only shows a pattern for a few
specific cases; orthogonality is not proven in general)



In [1]:
print("m, n, integral\n", 20*"-")
for n in range(0,4):
    for m in range(0,4):
        result = integrate(conjugate(phi(m)) * phi(n), (theta, 0, 2*pi))
        pprint((m, n, result))

## Finis

